In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
indep=dataset[["Gender_Male","Age",'EstimatedSalary']]
dep=dataset[["Purchased"]]

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import ComplementNB
param_grid={'alpha':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0], 'force_alpha':[True,False], 'fit_prior':[True,False], 'class_prior':[None], 'norm':[True,False]}
grid= GridSearchCV(ComplementNB(), param_grid, refit= True, verbose=3, n_jobs=-1, scoring= 'f1_weighted' )
grid.fit(X_train,y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


C:\Anaconda3run\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=ComplementNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,
                                   1.0],
                         'class_prior': [None], 'fit_prior': [True, False],
                         'force_alpha': [True, False], 'norm': [True, False]},
             scoring='f1_weighted', verbose=3)

In [10]:
y_pred=grid.predict(X_test)

In [11]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[40, 45],
       [20, 29]], dtype=int64)

In [12]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)

In [13]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.67      0.47      0.55        85
           1       0.39      0.59      0.47        49

    accuracy                           0.51       134
   macro avg       0.53      0.53      0.51       134
weighted avg       0.57      0.51      0.52       134



In [14]:
from sklearn.metrics import roc_auc_score
roc_score= roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])
roc_score

0.5834333733493396

In [15]:
print ( " the best parameter is ", grid.best_params_)

 the best parameter is  {'alpha': 0.1, 'class_prior': None, 'fit_prior': True, 'force_alpha': True, 'norm': False}


In [16]:
grid.cv_results_

{'mean_fit_time': array([0.01156912, 0.01436195, 0.01057339, 0.00977445, 0.00778499,
        0.00698166, 0.00857759, 0.00737944, 0.00718727, 0.00697975,
        0.00697608, 0.00777884, 0.00798426, 0.00758009, 0.00738173,
        0.00678167, 0.00658278, 0.00738101, 0.006182  , 0.00837851,
        0.00658402, 0.00737815, 0.00618377, 0.00598435, 0.0061831 ,
        0.00797825, 0.00818305, 0.00678291, 0.00837727, 0.00797911,
        0.00678167, 0.00618386, 0.00698142, 0.00638232, 0.00757909,
        0.0065824 , 0.00578461, 0.00758014, 0.00598965, 0.00618372,
        0.00718508, 0.00698218, 0.00757623, 0.00757976, 0.01037354,
        0.00618196, 0.00638165, 0.00777922, 0.00618382, 0.00678196,
        0.0085772 , 0.00757709, 0.00937381, 0.00598493, 0.00658245,
        0.00638494, 0.00797873, 0.00718064, 0.00638461, 0.00638361,
        0.00578294, 0.00778193, 0.00678158, 0.00718532, 0.00777955,
        0.00698133, 0.00598459, 0.00677958, 0.0081779 , 0.00658474,
        0.00698485, 0.00578256,

In [17]:
re=grid.cv_results_

In [18]:
table= pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_class_prior,param_fit_prior,param_force_alpha,param_norm,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011569,0.001620,0.025134,0.006226,0.1,None,True,True,True,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,41
1,0.014362,0.004746,0.014762,0.002707,0.1,None,True,True,False,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
2,0.010573,0.004067,0.012565,0.001350,0.1,None,True,False,True,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,41
3,0.009774,0.002706,0.013168,0.002128,0.1,None,True,False,False,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
4,0.007785,0.002630,0.012760,0.001718,0.1,None,False,True,True,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.007181,0.001465,0.010974,0.000006,1.0,None,True,False,False,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
76,0.005187,0.000977,0.010372,0.000488,1.0,None,False,True,True,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,41
77,0.007979,0.002274,0.016755,0.005256,1.0,None,False,True,False,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
78,0.007179,0.001935,0.012567,0.003431,1.0,None,False,False,True,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,41


In [19]:
grid.best_score_

0.5450916374836033

In [21]:
pre_input= ([[0,35,60000]])

In [22]:
result= grid.predict(pre_input)
result

C:\Anaconda3run\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but ComplementNB was fitted with feature names
  warnings.warn(


array([1], dtype=int64)